In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
E_B = 100
P_Battery_Max = 50
soc_upper_bound = 0.9
soc_floor_bound = 0.1

In [6]:


# 读取数据
data_A = pd.read_csv("data/园区A.csv")
data_B = pd.read_csv("data/园区B.csv")
data_C = pd.read_csv("data/园区C.csv")

# 23:00 的电池能不能给0:00用？0->23 or 6-5?
# 电池初始值？SOC(t=0)=?

def find_lack(data,h):
    for i in range(h+1,len(data)):
        if data[i] < 0:
            return i
    return -1

def Cal_charge(P_wind,data,h):
    if data['Curt_C'][h] >= 0:
        h_lack=find_lack(data['Curt_C'],h)
        if h_lack == -1:
            print(f'从{h}电后面没有负荷缺口')
            return 0
        else:
            extra=sum(data['Curt_C'][h:h_lack].apply(lambda x: P_Battery_Max if x > P_Battery_Max else x))
            lack=-data['Curt_C'][h_lack]

            while lack<extra and h_lack<len(data) and data['Curt_C'][h_lack]<=0:
                h_lack+=1
                lack+=(-data['Curt_C'][h_lack])

            if lack > extra:
                Bu_Wind=lack-extra
                if Bu_Wind>P_wind:
                    return P_wind
                else:
                    return Bu_Wind
            else:
                return 0 

def charge(d_P):
    global Battery
    tmp=Battery
    if P_Battery_Max >= d_P:
        if Battery + d_P >= init_Battery_capacity_upper_bound:
            Battery = init_Battery_capacity_upper_bound
        else:
            Battery += d_P
    else:
        if Battery + P_Battery_Max >= init_Battery_capacity_upper_bound:
            Battery = init_Battery_capacity_upper_bound
        else:
            Battery += P_Battery_Max
    #print('Battery充电:','delt_p:',d_P,' ',tmp,'->',Battery)
# 定义charge函数
def Process(P_user, P_light, P_wind,data,label,h):
    global Battery
    C_light=0
    C_wind=0 
    C_net=0
    if P_light > P_user:
        C_light = P_user
        P_user = 0
        P_light -= C_light
        charge(P_light)
    else:
        C_light = P_light
        P_user -= P_light
    
    if P_user == 0:
        return C_light, C_wind, C_net
    
    tmp=Battery
    if P_user < P_Battery_Max:
        if Battery-P_user<=init_Battery_capacity_floor_bound:
            Battery = init_Battery_capacity_floor_bound
            P_user -= Battery-init_Battery_capacity_floor_bound
        else:
            Battery -= P_user
            P_user = 0
            charge(P_wind)
    else:
        if Battery-P_Battery_Max<=init_Battery_capacity_floor_bound:
            Battery = init_Battery_capacity_floor_bound
            P_user -= Battery-init_Battery_capacity_floor_bound
        else:
            Battery -= P_Battery_Max
            P_user -= P_Battery_Max
    #print('Battery放电:',tmp,'->',Battery)
    if P_user == 0:
        return C_light, C_wind, C_net
    
    if P_wind > P_user:
        C_wind = P_user
        P_user = 0
        P_wind-=C_wind
        if label == 'B':
            charge(P_wind)
        if label == 'C':
            P_B=Cal_charge(P_wind,data,h)
            charge(P_B)
    else:
        C_wind = P_wind
        P_user -= P_wind
    
    if P_user > 0:
        C_net = P_user
        P_user = 0

    return C_light, C_wind, C_net





In [8]:
df_A = pd.DataFrame(columns=['E_B', 'P_Battery_Max', 'cost'])
df_B = pd.DataFrame(columns=['E_B', 'P_Battery_Max', 'cost'])
df_C = pd.DataFrame(columns=['E_B', 'P_Battery_Max', 'cost'])


while(E_B<=1000):
    P_Battery_Max = 50
    while(P_Battery_Max<=1000):
        # 初始化变量
        Battery = 0
        init_Battery_capacity_floor_bound = E_B * soc_floor_bound
        init_Battery_capacity_upper_bound = E_B * soc_upper_bound

        cost_of_Battery=(800*P_Battery_Max+1800*E_B)/3650

        # 处理数据
        #print('E_B:',E_B,'P_Battery_Max:',P_Battery_Max)
        # A处理
        n_rows = data_A.shape[0]

        # 创建一个长度与data_A行数相同的空列向量
        A_light = np.zeros((n_rows,1))
        A_wind = np.zeros((n_rows,1))
        A_net = np.zeros((n_rows,1))

        # 循环遍历data_A的每一行
        for index, row in data_A.iterrows():
            P_user = row['用电量']
            P_light = row['光伏供电量']
            P_wind = 0
            A_light[index],A_wind[index],A_net[index]=Process(P_user, P_light, P_wind,data_A,'A',index)

        #print(A_light)
        #print(A_wind)
        #print(A_net)
        cost_A=1*A_net+0.4*A_light
        #print('A成本:',sum(cost_A)+cost_of_Battery)


        #B处理
        n_rows = data_B.shape[0]

        # 创建一个长度与data_B行数相同的空列向量
        B_light = np.zeros((n_rows,1))
        B_wind = np.zeros((n_rows,1))
        B_net = np.zeros((n_rows,1))

        # 循环遍历data_B的每一行
        for index, row in data_B.iterrows():
            P_user = row['用电量']
            P_wind = row['风电供电量']
            P_light = 0
            B_light[index],B_wind[index],B_net[index]=Process(P_user, P_light, P_wind,data_B,'B',index)

        #print(B_light)
        #print(B_wind)
        #print(B_net)
        cost_B=1*B_net+0.5*B_wind
        #print('B风电购入',sum(B_wind))
        #print('B电网购入',sum(B_net))
        #print('B成本:',sum(cost_B)+cost_of_Battery)

        # C处理
        n_rows = data_C.shape[0]

        # 创建一个长度与data_C行数相同的空列向量
        C_light = np.zeros((n_rows,1))
        C_wind = np.zeros((n_rows,1))
        C_net = np.zeros((n_rows,1))

        data_C['Curt_C'] = data_C.iloc[:, 1] + data_C.iloc[:, 2] - data_C.iloc[:, 0]
        #print(data_C['Curt_C'])
        # 循环遍历data_C的每一行
        for index, row in data_C.iterrows():
            P_user = row['用电量']
            P_light = row['光伏供电量']
            P_wind = row['风电供电量']
            C_light[index],C_wind[index],C_net[index]=Process(P_user, P_light, P_wind,data_C,'C',index)

        #print(C_light)
        #print(C_wind)
        #print(C_net)
        cost_C=1*C_net+0.4*C_light+0.5*C_wind
        cost_A = np.array(cost_A)
        cost_A= cost_A.reshape(-1)
        cost_B = np.array(cost_B)
        cost_B= cost_B.reshape(-1)
        cost_C = np.array(cost_C)
        cost_C= cost_C.reshape(-1)
        #print('C成本:',sum(cost_C)+cost_of_Battery)
        #print('----------------------------------')
        new_row = pd.DataFrame({'E_B': [E_B], 'P_Battery_Max': [P_Battery_Max], 'cost': [sum(cost_A) + cost_of_Battery]})
        df_A = pd.concat([df_A, new_row], ignore_index=True)
        new_row = pd.DataFrame({'E_B': [E_B], 'P_Battery_Max': [P_Battery_Max], 'cost': [sum(cost_B) + cost_of_Battery]})
        df_B = pd.concat([df_B, new_row], ignore_index=True)
        new_row = pd.DataFrame({'E_B': [E_B], 'P_Battery_Max': [P_Battery_Max], 'cost': [sum(cost_C) + cost_of_Battery]})
        df_C = pd.concat([df_C, new_row], ignore_index=True)
        P_Battery_Max+=50
    
    E_B+=100
# 输出结果

df_A.to_excel('problem1/园区A_cost.xlsx')
df_B.to_excel('problem1/园区B_cost.xlsx')
df_C.to_excel('problem1/园区C_cost.xlsx')
# 画出E_B,P_Battery_Max与cost的关系
# 创建一个新的图形
# Create a 3D scatter plot
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] =False

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df_A['E_B'], df_A['P_Battery_Max'], df_A['cost'])

ax.set_xlabel('E_B')
ax.set_ylabel('P_Battery_Max')
ax.set_zlabel('cost')
plt.title('园区A E_B,P_Battery_Max与cost的关系')
plt.show()

# 保存图形到指定文件夹和文件名
output_folder = 'problem1/pictures'
output_filename = '园区A E_B,P_Battery_Max与cost的关系.png'
output_path = f'{output_folder}/{output_filename}'
plt.savefig(output_path, dpi=300)
plt.close()


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df_B['E_B'], df_B['P_Battery_Max'], df_B['cost'])

ax.set_xlabel('E_B')
ax.set_ylabel('P_Battery_Max')
ax.set_zlabel('cost')
plt.title('园区B E_B,P_Battery_Max与cost的关系')
plt.show()

# 保存图形到指定文件夹和文件名
output_folder = 'problem1/pictures'
output_filename = '园区B E_B,P_Battery_Max与cost的关系.png'
output_path = f'{output_folder}/{output_filename}'
plt.savefig(output_path, dpi=300)
plt.close()


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df_C['E_B'], df_C['P_Battery_Max'], df_C['cost'])

ax.set_xlabel('E_B')
ax.set_ylabel('P_Battery_Max')
ax.set_zlabel('cost')
plt.title('园区C E_B,P_Battery_Max与cost的关系')
plt.show()

# 保存图形到指定文件夹和文件名
output_folder = 'problem1/pictures'
output_filename = '园区C E_B,P_Battery_Max与cost的关系.png'
output_path = f'{output_folder}/{output_filename}'
plt.savefig(output_path, dpi=300)
plt.close()

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
# 1-2各园区分别配置 50kW/100kWh 储能， 
# 制定储能最优运行策略及购电计划，分析各园区运行经济性是否改善，
# 并解释其原因； 
# 1. 读取数据
# 2. 建立模型
# 3. 求解
# 4. 结果分析
